In [2]:
import bpy
import numpy as np
from IPython.display import display, Image
import os
import mathutils
from tqdm import tqdm
from typing import List, Dict
import bmesh
import cmath
import scipy.sparse
import scipy.linalg
from scipy.sparse.linalg import spsolve

import matplotlib.pyplot as plt

from line_profiler import profile

# install packages : "C:\Users\Pierre.Gilibert\OneDrive - ARVERNE\Bureau\blender-4.3.1-windows-x64\4.3\python\bin\python.exe" -m pip install scipy 

In [3]:
FOLDER = "C:\\Users\\pierr\\Documents\\Blender\\GeometryProcessing_Python\\"
FOLDER = "C:\\Users\\Pierre.Gilibert\\OneDrive - ARVERNE\\Documents\\Divers\\blender"
bpy.ops.wm.open_mainfile(filepath=os.path.join(FOLDER, "blender_notebook_v01.blend"))

{'FINISHED'}

In [6]:

class SectionIntegrals:
    """
    Class to compute integrals (Dirichlet and Mass integrals) using Chebyshev series and direct evaluations.
    """
    # Machine constants (IEEE double-precision)
    MACH = [
        2.2250738585072014e-308,  # Smallest positive number
        1.7976931348623157e+308,  # Largest representable number
        1.1102230246251565e-16,   # Machine epsilon
        2.2204460492503131e-16,   # Largest relative spacing
        3.0102999566398120e-01    # log10(2)
    ]

    # Chebyshev coefficients (Placeholder: fill with actual data)
    S11R = [ 0.0448875760891932036595562553276, 0.0278480909574822965157922173757, 0.00394490790249120295818107628687, -0.00157697939158619172562804651751, 
            -0.0000886578217796691901712579357311,0.0000301708056772263120428135787035, 9.521839632337438230089618156e-7,-3.00028307455805582080773625835e-7, 
            -6.14917009583473496433650831019e-9,1.85133588988085286010092653662e-9, 2.67848449041765751590373973224e-11,-7.82394575359355297437491915705e-12, 
            -8.44240072511090922609176843848e-14,2.41333276776166240844516922196e-14, 2.02015531985181413114834031833e-16,-5.68171271075270422851146478874e-17, 
            -3.80082421064644521052871349836e-19,1.05551739229841670238163200361e-19, 5.7758422925275435667221605993e-22,-1.58774695838716531303310462626e-22, 
            -7.24181766014636685673730787292e-25]  
    S11I = [0.100116671557942715638078149123,0.0429600096728215971268270800599, -0.00799014859477407505770275088389,-0.000664114111384495427035329182866, 
            0.000240714510952202000864758517061,9.89085259369337382687437812294e-6, -3.22040860178194578481012477174e-6,-8.08401148192350365282200249069e-8,
              2.48351290049260966544658921605e-8,4.24154988067028660399867468349e-10, -1.25611378629704490237955971836e-10,-1.56053077919196502557674988724e-12, 
              4.50565044006801278137904597946e-13,4.2641179237225098728291226479e-15, -1.2084245714879456268965803807e-15,-9.01338537885038989528688031325e-18, 
              2.5180796700698002962991581923e-18,1.51955263898294940481729370636e-20, -4.19737873024216866691628952458e-21,-2.092488792285595339755624521e-23, 
              5.72708467031136321701747126611e-24]
    S12R = [-0.376145877558191778393359413441,0.0775244431850198578126067647425, 0.0120396593748540634695397747695,-0.00385683684390247509721340352427, 
            -0.000232359275790231209370627606991,0.0000697318379146209092637310696007, 2.32354473986257272021507575389e-6,-6.71692140309360615694979580992e-7, 
            -1.43946361256617673523038166877e-8,4.06087820907414336567714443732e-9, 6.10183339004616075548375321861e-11,-1.69196418769523832825063863136e-11, 
            -1.88669746820541798989965091628e-13,5.16473095452962111184823547686e-14, 4.45066881692009291504139737861e-16,-1.20625107617859803735741992452e-16, 
            -8.28193837331508300767103116139e-19,2.22680015825230528892642524445e-19, 1.24755889505424049389100515561e-21,-3.33254971913153176741833960484e-22,
              -1.55307002839777371508497520751e-24]
    S12I = [0.0527472790869782317601048210983,0.00823962722148093961886198320927, -0.0205185842051817330153151013327,-0.00184683218270819613487368071941, 
            0.000569681886932212757533488372406,0.0000248774530818801164177266528608, -7.31121019876580624171992432347e-6,-1.92744564223806538367454388776e-7, 
            5.49794278719049727550379096876e-8,9.78237385539447442446850072421e-10, -2.7341624177723508216430132999e-10,-3.51839815887772323640101921381e-12, 
            9.68934411607055794052256859665e-13,9.45703963505047353201918875825e-15, -2.57516976113400217760868402425e-15,-1.97419921753098238455550504742e-17, 
            5.32820017906655555903355375475e-18,3.29581793797656865402793252539e-20, -8.83137325823594007269279476114e-21,-4.50279718100548728336329365981e-23, 
            1.19941679774924468309434420379e-23]
    M12R = [0.148523151773238914750879360089,-0.0117856118001224048185631301904, -0.00248887208039014371691400683052,0.000250045060357076469386198883676, 
            0.0000227217776065076434637230864113,-2.48764935230787745662127026799e-6, -1.32138506847814502856384193414e-7,1.50966754393693942843767293542e-8, 
            5.3472999553162661403204445045e-10,-6.26136041009708550772228055719e-11, -1.59574066624737000616598104732e-12,1.89788785691219687197167013023e-13, 
            3.66030609080549274006207730375e-15,-4.39955659500182569051978906011e-16, -6.65848768159000092224193226014e-18,8.06343127453005031535923212263e-19, 
            9.84397490339224661524630997726e-21,-1.19869887155210161836484730378e-21, -1.20634550494837590549640883469e-23,1.47512193662595435067359954287e-24, 
            1.24549093756962710863096766634e-26]
    M12I = [-0.0454399665519585306943416687117,-0.0210517666740874019203591488894, 0.00194647501081621201871675259482,0.000253466068123907163346571754613, 
            -0.0000268083453427538717591876419304,-1.82138740336918117478832696004e-6, 2.04357511048425337951376869602e-7,8.75944656915074206478854298947e-9, 
            -1.01466837126303146739791005703e-9,-3.02573132377805421636557302451e-11, 3.57358222114420372764650037191e-12,7.88121312149152771558608913996e-14, 
            -9.42758576193708862552405242331e-15,-1.60439904050827900099939709069e-16, 1.93624791035947590366500765061e-17,2.62394448214143482490534256935e-19, 
            -3.18700789496399461681365308408e-20,-3.52400207248027768109209530864e-22, 4.30074555255053206057921088056e-23,3.95655079023456015736315286131e-25, 
            -4.84642137915095135859812028886e-26]


    def __init__(self):
        # Number of terms initialized for each Chebyshev series
        self.ns11r = self.ns11i = 0
        self.ns12r = self.ns12i = 0
        self.nm12r = self.nm12i = 0

    def inits(self, series, eta):
        """
        Determine the number of terms required to achieve precision `eta`.
        """
        err = 0.0
        n = len(series)
        for i in range(n - 1, -1, -1):
            err += abs(series[i])
            if err > eta:
                return i + 1
        return 0

    def csevl(self, x, cs, n):
        """
        Evaluate Chebyshev series at normalized value x in [-1, 1].
        """
        b0, b1, b2 = 0.0, 0.0, 0.0
        twox = 2 * x
        for i in range(n - 1, -1, -1):
            b2, b1 = b1, b0
            b0 = twox * b1 - b2 + cs[i]
        return (b0 - b2) / 2

    def s11(self, t):
        """
        Compute s11 integral using Chebyshev series.
        """
        if not self.ns11r:
            self.ns11r = self.inits(self.S11R, self.MACH[2] / 10)
            self.ns11i = self.inits(self.S11I, self.MACH[2] / 10)
        real = self.csevl(t, self.S11R, self.ns11r)
        imag = self.csevl(t, self.S11I, self.ns11i)
        return complex(real, imag)

    def s12(self, t):
        """
        Compute s12 integral using Chebyshev series.
        """
        if not self.ns12r:
            self.ns12r = self.inits(self.S12R, self.MACH[2] / 10)
            self.ns12i = self.inits(self.S12I, self.MACH[2] / 10)
        real = self.csevl(t, self.S12R, self.ns12r)
        imag = self.csevl(t, self.S12I, self.ns12i)
        return complex(real, imag)

    def dirichlet_ii(s, gjj, gjk, gkk):
        return .25 * ((gjj - 2 * gjk + gkk)+ s * s * (gjj + gjk + gkk) / 90)

    def dirichlet_ij_direct(self, s, gii, gij, gjj):
        """
        Direct evaluation of Dirichlet integral for large s.
        """
        s2 = s * s
        _is = 1j*s
        _is3 = _is * s2
        s4 = s2 * s2
        eis = np.cos(s) + 1j * np.sin(s)
        return ((3*gii + 4*gij + 3*gjj) + _is*(gii + gij + gjj) - _is3*gij/6 + eis*(-(3*gii + 4*gij + 3*gjj) + _is*(2*gii + 3*gij + 2*gjj) + s2*(gii + 2*gij + gjj)/2))/s4 + (gii - 2*gij + gjj)/24 - _is*(gii - 2*gij + gjj)/60

        return complex(0.0, 0.0)  # Replace with actual direct computation

    def dirichlet_ij(self, s, gii, gij, gjj):
        """
        Compute Dirichlet integral.
        """
        if abs(s) > np.pi:
            return self.dirichlet_ij_direct(s, gii, gij, gjj)
        t = s * 2 / np.pi - 1 if s > 0 else -s * 2 / np.pi - 1
        result = ((gii + gjj) * self.s11(t) + gij * self.s12(t))
        return result.conjugate() if s > 0 else result

    def mass_ii(self):
        """
        Compute MassII integral.
        """
        return 1 / 6.0

    def mass_ij_direct(self, s):
        """
        Direct evaluation of MassIJ integral for large s.
        """
        s2 = s * s
        s4 = s2 * s2
        is_ = complex(0, s)
        is3 = complex(0, s * s2)
        return (6 * np.exp(is_) - 6 - 6 * is_ + 3 * s2 + is3) / (3 * s4)

    def mass_ij(self, s):
        """
        Compute MassIJ integral.
        """
        if abs(s) > np.pi:
            return self.mass_ij_direct(s)
        t = s / np.pi * 2 - 1 if s > 0 else -s / np.pi * 2 - 1
        return self.s12(t).conjugate() if s > 0 else self.s12(t)

In [7]:

def rotate_coord_system_vectorized(uv, vv, nf, nv):
    dot = np.sum(nv*nf, axis=1)
    dot_neg = np.where(dot <= -1)[0]
    dot = dot[:,None]
    perp = nf - dot * nv
    dperp = (nf + nv)/(1+dot)
    new_uv = uv - dperp * np.sum(perp*uv, axis=1)[:,None] # u - dperp * (perp.u)
    new_vv = vv - dperp * np.sum(perp*vv, axis=1)[:,None]

    new_uv[dot_neg]=-uv[dot_neg]
    new_uv[dot_neg]=-vv[dot_neg]

    return new_uv, new_vv

def project_curvature_tensor_vectorized(uf, vf, nf, old_ku, old_kuv, old_kv, up, vp, n):
    """
    Perform a projection of the tensor variables to the vertex coordinate system.
    
    Parameters:
    uf, vf (numpy.ndarray): Face coordinate system unit vectors.
    nf (numpy.ndarray): Normal vector of the face.
    old_ku, old_kuv, old_kv (float): Face curvature tensor variables.
    up, vp (numpy.ndarray): Vertex coordinate system unit vectors.
    n (numpy.ndarray): Normal vector of the vertex.
    
    Returns:
    new_ku, new_kuv, new_kv (float): Vertex curvature tensor variables.
    """
    def compute_coeff(a, b, ku, kuv, kv, c, d):
        return ku * c * a + kuv * (a*d + b*c) + kv * d * b # [a,b] @ [[ku, kuv], [kuv, kv]] @ [[c], [d]]
    
    # Rotate the coordinate system
    r_new_u, r_new_v = rotate_coord_system_vectorized(up, vp, nf, n)

    u1, u2 = np.sum(r_new_u*uf, axis=1), np.sum(r_new_v*uf, axis=1)
    v1, v2 = np.sum(r_new_u*vf, axis=1), np.sum(r_new_v*vf, axis=1)

    new_ku = compute_coeff(u1, v1, old_ku, old_kuv, old_kv, u1, v1)
    new_kuv = compute_coeff(u1, v1, old_ku, old_kuv, old_kv, u2, v2)
    new_kv = compute_coeff(u2, v2, old_ku, old_kuv, old_kv, u2, v2)

    return new_ku, new_kuv, new_kv



def compute_mass_ij_direct(s):
    s2 = s*s
    s4 = s2*s2
    ejs = np.cos(s) + 1j*np.sin(s) # faster than np.exp(1j*s)
    return (6*ejs - 6 - 6j*s + 3 * s2 + 1j*s*s2)/(3*s4)

def compute_dirichlet_ii(s, gjj, gjk, gkk):
  # gjj = |e_{ij}|^2, gjk = <e_{ij}, e_{ki}>, gkk = |e_{ki}|^2
  return .25 * ((gjj - 2 * gjk + gkk)+ s * s * (gjj + gjk + gkk) / 90)

def compute_dirichlet_ij_direct(s, gii, gij, gjj):
    # gii = |e_{ki}|^2, gij = <e_{ki}, e_{kj}>, gjj = |e_{jk}|^2
    s2 = s*s
    s4 = s2*s2
    ejs = np.cos(s) + 1j*np.sin(s) # faster than np.exp(1j*s)
    _is = 1j*s
    is3 = _is * s2
    return (((3*gii + 4*gij + 3*gjj) + _is*(gii + gij + gjj) - is3*gij/6
      + ejs*(-(3*gii + 4*gij + 3*gjj)
				      + _is*(2*gii + 3*gij + 2*gjj)
				      + s2*(gii + 2*gij + gjj)/2))/s4
     + (gii - 2*gij + gjj)/24 - _is*(gii - 2*gij + gjj)/60)



class HEEdge:
    def __init__(self, edge, face, orientation:int):
        self.edge = edge
        self.face = face
        self.orientation = orientation
        self.vertex = edge.verts[orientation] # orientation in {0,1} stating whether the self.edge is in the same (0) direction as self or the opposite (1) direction.
        self.next = None
        self.twin = None
        self.angle_from_X = 0.0
        self.transport_coeff = 0.0

    def __str__(self):
        return f"HEedge : {self.edge} - orientation: {self.orientation}"
    
    def __repr__(self):
        return f"HEedge : {self.edge} - orientation: {self.orientation}"

    def __getattr__(self, item):
        # Delegate attribute access to the internal bmesh instance
        return getattr(self.edge, item)
    

class MYMesh:
    def __init__(self):
        self.bm = bmesh.new()
        self.facecorners = []
        self.vert2facecorner = {}
        self.facecorner_attributes = {}             # facecorners attributes
        self.vertex_attributes = {"u":{}, "v":{}}   # u,v : basis vector of the tangent plane at each vertex
        self.edge_attributes = {}
        self.face_attributes = {}
        self.face_vertex_to_facecorner = {}
        self.co = None                              # A (|V|, 3) np array where row i contains the x,y,z coordinates of the vertex indexed by i 
        self.cotan = None                           # A (3|F|,) np array where entry i is the cotangent of the angle of facecorner (loop) i
        self.face_areas = None                      # A (|F|,) np array where entry i is the area for face i
        self.internal_angles = None                 # A (3|F|,) np array where entry i is the angle of facecorner (loop) i
        self.fv = None                              # A (|F|, 3) np array where row i contains the indices of the vertices of face i
        self.heedges = []                           # List of Half edges
        self.dict_vert2heedges = {}


    def from_mesh(self, mesh_data):
        """ Mimic the bmesh from_mesh function. """
        self.bm.from_mesh(mesh_data)
        self.vert2facecorner = {v:[] for v in self.verts}
        self.facecorners = []
        self.facecorner_attributes["area"] = {}
        for face in self.faces:
            self.face_vertex_to_facecorner[face] = {}
            for loop in face.loops:
                self.vert2facecorner[loop.vert].append(loop)
                self.facecorners.append(loop)
                self.facecorner_attributes["area"][loop] = 0 # init 0 area
                self.face_vertex_to_facecorner[face][loop.vert] = loop
            
        self.fv = np.array([[fc.vert.index for fc in f.loops] for f in self.faces], dtype=int) # face #f has vertex [vi, vj, vk]
        

    def to_mesh(self, mesh_data):
        """ Mimic the bmesh to_mesh function. """
        self.bm.to_mesh(mesh_data)
    
    # Add any other bmesh methods as needed:
    def free(self):
        """ Mimic the bmesh free function. """
        self.bm.free()

    # Add a custom method to directly access the internal bmesh:
    def __getattr__(self, item):
        # Delegate attribute access to the internal bmesh instance
        return getattr(self.bm, item)
    
    def ensure_lookup_tables(self):
        self.verts.ensure_lookup_table()
        self.edges.ensure_lookup_table()
        self.faces.ensure_lookup_table()

    def create_halfedge_datastructure(self):
        self.dict_vert2heedges = {}
        # self.edge_attributes["heedges"] = {}
        # for e in self.edges:
        #     self.edge_attributes["heedges"][e] = []
        for f in self.faces:
            if len(f.verts) != 3:
                raise ValueError("Not a triangular mesh : triangulate it beforehand !!")
            v_orientation = [f.verts[0].index, f.verts[1].index, f.verts[2].index]
            face_heedges = []
            for e in f.edges:
                # e0, e1, e2 = f.edges[0], f.edges[1], f.edges[2]
                v0, v1 = v_orientation.index(e.verts[0].index), v_orientation.index(e.verts[1].index)
                orientation = 1 - ((abs(v0-v1) == 1 and v0 < v1) or (abs(v0-v1) == 2 and v0 > v1)) # 0 if the edge matches the ccw convention of the face, 1 otherwise
                # print(e, v_orientation.index(e.verts[0].index), v_orientation.index(e.verts[1].index), orientation)
                face_heedges.append(HEEdge(e, f, orientation))
            he0, he1, he2 = face_heedges
            he0.next = he1
            he1.next = he2
            he2.next = he0
            self.dict_vert2heedges[(he0.vertex.index, he1.vertex.index)] = he0
            self.dict_vert2heedges[(he1.vertex.index, he2.vertex.index)] = he1
            self.dict_vert2heedges[(he2.vertex.index, he0.vertex.index)] = he2
            self.heedges.append(he0)
            self.heedges.append(he1)
            self.heedges.append(he2)
            # self.edge_attributes["heedges"][he0.edge] = he0
            # self.edge_attributes["heedges"][he1.edge] = he1
            # self.edge_attributes["heedges"][he2.edge] = he2

        for k in self.dict_vert2heedges:
            i0, i1 = k
            if i0 < i1:
                continue
            kr = (i1, i0)
            if kr in self.dict_vert2heedges:
                self.dict_vert2heedges[k].twin = self.dict_vert2heedges[kr]
                self.dict_vert2heedges[kr].twin = self.dict_vert2heedges[k]
        
        boundary_hedges = []
        for e in self.edges:
            if not e.is_boundary:
                continue
            i0, i1 = e.verts[0].index, e.verts[1].index
            if (i0, i1) in self.dict_vert2heedges:
                existing_he = self.dict_vert2heedges[(i0, i1)]
            elif (i1, i0) in self.dict_vert2heedges:
                existing_he = self.dict_vert2heedges[(i1, i0)]
            else:
                raise ValueError(f"Unable to find an half edge between vertices {i0} and {i1}")
            new_he = HEEdge(e, None, 1-existing_he.orientation)
            existing_he.twin = new_he
            new_he.twin = existing_he
            boundary_hedges.append(new_he)
            self.dict_vert2heedges[(new_he.vertex.index, e.other_vert(new_he.vertex).index)] = new_he
            self.heedges.append(new_he)

        # match the boundary hedges
        num_link_to_create = len(boundary_hedges)
        num_link_created = 0
        while num_link_created != num_link_to_create:
            he = boundary_hedges[num_link_created]
            # other_index = -1
            for i, ohe in enumerate(boundary_hedges):
                if ohe.vertex == he.twin.vertex:
                    other_index = i
                    break
            # ohe = boundary_hedges.pop(other_index)
            he.next = ohe
            num_link_created+=1


        # verification : all hedges should have a twin and a next defined
        for k in self.dict_vert2heedges:
            he = self.dict_vert2heedges[k]
            if he.twin is None or he.next is None:
                raise ValueError(f"Unable to define a twin or a next for hedge {k} : {he}")



    def _calculate_corner_area(self):
        nfaces = len(self.faces)
        ffc = np.arange(3*nfaces).reshape((nfaces, 3)) # face corner index : face #f has corners [i, j, k];
        angles = self.internal_angles[ffc] # array of alpha_i, alpha_j, alpha_k
        eij = self.co[self.fv[:,1]] - self.co[self.fv[:,0]] # edges
        ejk = self.co[self.fv[:,2]] - self.co[self.fv[:,1]]
        eki = self.co[self.fv[:,0]] - self.co[self.fv[:,2]]
        lij2 = eij[:,0]*eij[:,0] + eij[:,1]*eij[:,1] + eij[:,2]*eij[:,2] # squared lengths of the edges
        ljk2 = ejk[:,0]*ejk[:,0] + ejk[:,1]*ejk[:,1] + ejk[:,2]*ejk[:,2]
        lki2 = eki[:,0]*eki[:,0] + eki[:,1]*eki[:,1] + eki[:,2]*eki[:,2]

        # in case triangle i,j,k is non obtuse (all its angles are < pi/2) then this is the area of each face corner
        non_obtuse_area_i = 0.125 * (lij2 * self.cotan[ffc[:,2]] + lki2 * self.cotan[ffc[:,1]])
        non_obtuse_area_j = 0.125 * (ljk2 * self.cotan[ffc[:,0]] + lij2 * self.cotan[ffc[:,2]])
        non_obtuse_area_k = 0.125 * (lki2 * self.cotan[ffc[:,1]] + ljk2 * self.cotan[ffc[:,0]])

        facecorners_areas = np.zeros(3*nfaces).reshape((nfaces, 3))
        # check whether the angles are less than pi/2
        small_angle_bool = angles < np.pi/2
        big_angle_bool = ~small_angle_bool
        # True for all corners of triangle i,j,k ==> use the non_obtuse_area
        non_obtuse_bool = np.logical_and(small_angle_bool[:,0], np.logical_and(small_angle_bool[:,1], small_angle_bool[:,2]))
        facecorners_areas[non_obtuse_bool, 0] = non_obtuse_area_i[non_obtuse_bool]
        facecorners_areas[non_obtuse_bool, 1] = non_obtuse_area_j[non_obtuse_bool]
        facecorners_areas[non_obtuse_bool, 2] = non_obtuse_area_k[non_obtuse_bool]
        # False for corner p in {i,j,k}: use half the face area for p and a quarter for the two others corners
        facecorners_areas[big_angle_bool[:,0]] = self.face_areas[big_angle_bool[:,0]][:,None] * np.array([[0.5, 0.25, 0.25]])
        facecorners_areas[big_angle_bool[:,1]] = self.face_areas[big_angle_bool[:,1]][:,None] * np.array([[0.25, 0.5, 0.25]])
        facecorners_areas[big_angle_bool[:,2]] = self.face_areas[big_angle_bool[:,2]][:,None] * np.array([[0.25, 0.25, 0.5]])
        self.facecorner_attributes["area"] = facecorners_areas.ravel()

    def _calculate_corner_angles_and_face_areas(self):
        self.co = np.array([v.co for v in self.verts])

        vi = self.co[self.fv[:,0]]
        vj = self.co[self.fv[:,1]]
        vk = self.co[self.fv[:,2]]

        eij, ejk, eki = vj-vi, vk-vj, vi-vk
        lij2 = eij[:,0]*eij[:,0] + eij[:,1]*eij[:,1] + eij[:,2]*eij[:,2]
        ljk2 = ejk[:,0]*ejk[:,0] + ejk[:,1]*ejk[:,1] + ejk[:,2]*ejk[:,2]
        lki2 = eki[:,0]*eki[:,0] + eki[:,1]*eki[:,1] + eki[:,2]*eki[:,2]
        lij, ljk, lki = np.sqrt(lij2), np.sqrt(ljk2), np.sqrt(lki2)

        s = 0.5 * (lij + ljk + lki) # half perimeter of every triangle
        self.face_areas = 0.5 * np.sqrt(s * (s - lij) * (s - ljk) * (s - lki)) # Heron's formula for the area of the triangles
        
        q_i = -ljk2 + lij2 + lki2
        q_j = -lki2 + ljk2 + lij2
        q_k = -lij2 + lki2 + ljk2

        denom_inv = 1/(8*self.face_areas)
        self.cotan = np.zeros(3*len(bm.faces))
        self.cotan[0::3] = q_i*denom_inv
        self.cotan[1::3] = q_j*denom_inv
        self.cotan[2::3] = q_k*denom_inv

        self.internal_angles = np.zeros(3*len(bm.faces))
        self.internal_angles[0::3] = np.arccos(np.clip(q_i / (2*lij * lki), -1, 1))
        self.internal_angles[1::3] = np.arccos(np.clip(q_j / (2*ljk * lij), -1, 1))
        self.internal_angles[2::3] = np.arccos(np.clip(q_k / (2*lki * ljk), -1, 1))

    def calculate_required_data(self):

        self._calculate_corner_angles_and_face_areas()
        self._calculate_corner_area()
        self._calculate_vertex_area()
        self._calculate_vertex_basis()

    def _calculate_vertex_area(self):
        # self.vertex_attributes["area"] = np.array([np.sum([self.facecorner_attributes["area"][fc.index] for fc in self.vert2facecorner[v]]) for v in self.verts])
        val = []
        for v in self.verts:
            s = 0
            for fc in self.vert2facecorner[v]:
                s+= self.facecorner_attributes["area"][fc.index]
            val.append(s)
        self.vertex_attributes["area"] = np.array(val)
            
    def _calculate_vertex_basis(self):
        eij = np.array([v.link_edges[0].other_vert(v).co - v.co for v in bm.verts])

        n = np.array([v.normal for v in self.verts])
        u = eij - np.sum(eij*n, axis=1)[:,None]
        u = u/np.linalg.norm(u, axis=1)[:,None]
        v = np.cross(n, u)
        v = v/np.linalg.norm(v, axis=1)[:,None]

        self.vertex_attributes["u"] = u
        self.vertex_attributes["v"] = v

    def _custom_solve_Ab_vectorized(self, a, b, c, d, e, f, vec_b):
        """
        Heavily optimised method to solve for Ax = vec_b (least square) (about 45x faster than scipy.sparse.linalg.lsqr(A, vec_b)). A is of the form:
        A = [
        A_block_0, 0, 0, ...
        0, A_block_1, 0, ...
        0, 0, A_block_2, ...
        ]
        with
        A_block_i = [
        [a[i], b[i], 0],
        [0, a[i], b[i]],
        [c[i], d[i], 0],
        [0, c[i], d[i]],
        [e[i], f[i], 0],
        [0, e[i], f[i]]
        ]
        One knows that min||Ax-vec_b||^2 is obtained for x = ((A.T @ A)^{-1}) @ A.T @ vec_b.
        Let's first compute (A.T @ A)^{-1} (AtAinv in the code)
        then A.T @ vec_b (Atvec_b in the code)
        before returning x
        """
        aa, bb, cc, dd, ee, ff = a*a, b*b, c*c, d*d, e*e, f*f
        aaa, bbb, ccc, ddd, eee, fff = aa*a, bb*b, cc*c, dd*d, ee*e, ff*f
        aaaa, bbbb, cccc, dddd, eeee, ffff = aaa*a, bbb*b, ccc*c, ddd*d, eee*e, fff*f

        denom = aaaa*dd + aaaa*ff - 2*aaa*b*c*d - 2*aaa*b*e*f + aa*bb*cc + aa*bb*dd + aa*bb*ee + aa*bb*ff + aa*cc*dd + 2*aa*cc*ff - 2*aa*c*d*e*f + aa*dddd + 2*aa*dd*ee + 2*aa*dd*ff + aa*ee*ff + aa*ffff - 2*a*bbb*c*d - 2*a*bbb*e*f - 2*a*b*ccc*d - 2*a*b*cc*e*f - 2*a*b*c*ddd - 2*a*b*c*d*ee - 2*a*b*c*d*ff - 2*a*b*dd*e*f - 2*a*b*eee*f - 2*a*b*e*fff + bbbb*cc + bbbb*ee + bb*cccc + bb*cc*dd + 2*bb*cc*ee + 2*bb*cc*ff - 2*bb*c*d*e*f + 2*bb*dd*ee + bb*eeee + bb*ee*ff + cccc*ff - 2*ccc*d*e*f + cc*dd*ee + cc*dd*ff + cc*ee*ff + cc*ffff - 2*c*ddd*e*f - 2*c*d*eee*f - 2*c*d*e*fff + dddd*ee + dd*eeee + dd*ee*ff
        m00 = aa*dd + aa*ff - 2*a*b*c*d - 2*a*b*e*f + bbbb + bb*cc + 2*bb*dd + bb*ee + 2*bb*ff + cc*ff - 2*c*d*e*f + dddd + dd*ee + 2*dd*ff + ffff
        m01 = -a*bbb - a*b*dd - a*b*ff - bb*c*d - bb*e*f - c*ddd - c*d*ff - dd*e*f - e*fff
        m02 = aa*bb + 2*a*b*c*d + 2*a*b*e*f + cc*dd + 2*c*d*e*f + ee*ff
        # m10 = m01
        m11 = aa*bb + aa*dd + aa*ff + bb*cc + bb*ee + cc*dd + cc*ff + dd*ee + ee*ff
        m12 = -aaa*b - aa*c*d - aa*e*f - a*b*cc - a*b*ee - ccc*d - cc*e*f - c*d*ee - eee*f
        # m20 = m02
        # m21 = m12
        m22 = aaaa + 2*aa*cc + aa*dd + 2*aa*ee + aa*ff - 2*a*b*c*d - 2*a*b*e*f + bb*cc + bb*ee + cccc + 2*cc*ee + cc*ff - 2*c*d*e*f + dd*ee + eeee

        nvertex = denom.shape[0]
        denom_inv = 1./denom
        m00, m01, m02, m11, m12, m22 = m00*denom_inv, m01*denom_inv, m02*denom_inv, m11*denom_inv, m12*denom_inv, m22*denom_inv
        val = np.zeros(9*nvertex)
        val[0::9] = m00
        val[1::9] = m01
        val[2::9] = m02
        val[3::9] = m01
        val[4::9] = m11
        val[5::9] = m12
        val[6::9] = m02
        val[7::9] = m12
        val[8::9] = m22
        row = np.repeat(np.arange(3*nvertex), 3)
        col = np.array([0,1,2]*(3*nvertex)) + 3*np.repeat(np.arange(nvertex), 9)

        AtAinv = scipy.sparse.coo_array((val, (row, col))).tocsr()
  
        Atvec_b = np.zeros(3*nvertex)
        Atvec_b[0::3] = a * vec_b[0::6] + c * vec_b[2::6] + e * vec_b[4::6]
        Atvec_b[1::3] = b * vec_b[0::6] + a * vec_b[1::6] + d * vec_b[2::6] + c * vec_b[3::6] + f * vec_b[4::6] + e * vec_b[5::6]
        Atvec_b[2::3] = b * vec_b[1::6] + d * vec_b[3::6] + f * vec_b[5::6]

        x = AtAinv.dot(Atvec_b) # Ax = vec_b least square : min||Ax - vec_b||^2 ==> x = (A.T * A)^{-1} * A.T * vec_b

        return x

    def calculate_curvature_vectorized(self, use_optimized_solve=True):

        nface = len(self.faces)
        nvertex = len(self.verts)

        # co = np.array([v.co for v in self.verts])
        nv = np.array([v.normal for v in self.verts])
        nf = np.array([f.normal for f in self.faces])
        # fv = np.array([[v.index for v in f.verts] for f in self.faces])

        uf = self.co[self.fv[:,0]]-self.co[self.fv[:,1]]
        uf = uf/np.linalg.norm(uf, axis=1)[:,None]
        vf = np.cross(nf, uf)

        vi, vj, vk = self.co[self.fv[:,0]], self.co[self.fv[:,1]], self.co[self.fv[:,2]]
        ni, nj, nk = nv[self.fv[:,0]], nv[self.fv[:,1]], nv[self.fv[:,2]]
        
        ejk, eki, eij = vk-vj, vi-vk, vj-vi
        njk, nki, nij = nk-nj, ni-nk, nj-ni

        a, b = np.einsum('ij,ij->i', ejk, uf), np.einsum('ij,ij->i', ejk, vf)
        c, d = np.einsum('ij,ij->i', eki, uf), np.einsum('ij,ij->i', eki, vf)
        e, f = np.einsum('ij,ij->i', eij, uf), np.einsum('ij,ij->i', eij, vf)

        vec_b = np.zeros(6*nface)
        vec_b[0::6] = np.einsum('ij,ij->i', njk, uf)
        vec_b[1::6] = np.einsum('ij,ij->i', njk, vf)
        vec_b[2::6] = np.einsum('ij,ij->i', nki, uf)
        vec_b[3::6] = np.einsum('ij,ij->i', nki, vf)
        vec_b[4::6] = np.einsum('ij,ij->i', nij, uf)
        vec_b[5::6] = np.einsum('ij,ij->i', nij, vf)

        if not use_optimized_solve:
            value = np.vstack((a,b, a,b, c,d, c,d, e,f, e,f)).T
            row = np.repeat(np.arange(6*nface),2)
            col = np.tile([0, 1, 1, 2], 3*len(bm.faces)).reshape((-1,4)) + 3*np.repeat(np.arange(len(bm.faces)), 3).reshape((-1,1))
            value = value.ravel()
            col = col.ravel()
            A = scipy.sparse.coo_array((value, (row, col))).tocsr()
            x = scipy.sparse.linalg.lsqr(A, vec_b)[0] # only return the solution
        else:
            x = self._custom_solve_Ab_vectorized(a, b, c, d, e, f, vec_b) # Heavily optimized code to speed up the computation : about 45x faster than scipy.sparse.linalg.lsqr(A, vec_b)
        # project X into the vertex frame
        weights = self.facecorner_attributes["area"] / self.vertex_attributes["area"][self.fv.ravel()]
        fv_ravel = self.fv.ravel()
        uv = self.vertex_attributes["u"]
        vv = self.vertex_attributes["v"]
        uv_dup = uv[fv_ravel]
        vv_dup = vv[fv_ravel]

        nv_dup = nv[fv_ravel].reshape((3*self.fv.shape[0], 3))
        nf_dup = np.tile(nf,3).reshape((3*nface, 3))
        uf_dup = np.tile(uf,3).reshape((3*nface, 3))
        vf_dup = np.tile(vf,3).reshape((3*nface, 3))
        x_dup = np.tile(x.reshape((nface, 3)), 3).reshape((3*nface, 3))
        new_ku, new_kuv, new_kv = project_curvature_tensor_vectorized(uf_dup, vf_dup, nf_dup, x_dup[:,0], x_dup[:,1], x_dup[:,2], uv_dup, vv_dup, nv_dup)
        # new_k is on the facecorner domain. weight it to express it on the vertex domain
        # Build a matrix of mass so that M * new_k is on the vertex domain
        row, col, val = [], [], []
        for v in self.verts:
            for fc in self.vert2facecorner[v]:
                row.append(v.index)
                col.append(fc.index)
                val.append(weights[fc.index])

        M = scipy.sparse.coo_array((val, (row, col))).tocsr()
        ku = M.dot(new_ku)
        kuv = M.dot(new_kuv)
        kv = M.dot(new_kv)
        # sfm = np.vstack((ku, kuv, kv)).T # curvature matrix is [[ku, kuv], [kuv, kv]] for each [ku, kuv, kv] in sfm

        c, s, tt = np.ones(nvertex), np.zeros(nvertex), np.zeros(nvertex) 
        h = 0.5 * (kv-ku)/kuv
        root = np.sqrt(1 + h*h)
        hneg = h<0
        hpos = ~hneg
        tt[hneg] = 1/(h[hneg]-root[hneg])
        tt[hpos] = 1/(h[hpos]+root[hpos])
        c = 1 / np.sqrt(1 + tt * tt)
        s = tt * c
        ttkuv = tt * kuv
        k1 = ku - ttkuv
        k2 = kv + ttkuv
        kuv0 = kuv == 0
        k1[kuv0] = ku[kuv0]
        k2[kuv0] = kv[kuv0]
        k1lk2 = np.abs(k1) < np.abs(k2)
        # abs(k1) > abs(k2) ==> e1 = cos * uv - sin * vv
        # else ==> e1 = sin * uv + cos * vp
        c, s = c[:,None], s[:,None]
        e1 = c * uv - s * vv
        e1[k1lk2] = s[k1lk2]*uv[k1lk2] + c[k1lk2] * vv[k1lk2]
        e2 = np.cross(nv, e1)
        e2 = e2 / np.linalg.norm(e2, axis=1)[:,None]
        return k1, k2, e1, e2

    def assign_distinguished_vector_X(self):
        all_X = []
        for v in self.verts:
            if v.is_boundary:
                boundary_edges = [e for e in v.link_edges if e.is_boundary]
                if len(boundary_edges) != 2:
                    raise ValueError(f"Non manifold mesh : vertex {v.index} does not have exactly 2 adjacent boundary edges")
                other_vs = [boundary_edges[0].other_vert(v), boundary_edges[1].other_vert(v)]
                # select the hedge starting at v and having a non None face
                vector_X = None
                for ov in other_vs:
                    key = (v.index, ov.index)
                    key_r = (ov.index, v.index)
                    for k in [key, key_r]:
                        he = self.dict_vert2heedges[k]
                        if he.vertex == v and he.face is not None:
                            vector_X = he
                            break
                    if vector_X is not None:
                        break
                if vector_X is None:
                    raise ValueError(f"Unable to find an X for boundary vertex {v.index}")
                all_X.append(vector_X)
                # if (v.index, boundary_edges[0].other_vert(v).index) in self.dict_vert2heedges:
                #     all_X.append(self.dict_vert2heedges[(v.index, boundary_edges[0].other_vert(v).index)])
                # elif (v.index, boundary_edges[1].other_vert(v).index) in self.dict_vert2heedges:
                #     all_X.append(self.dict_vert2heedges[(v.index, boundary_edges[1].other_vert(v).index)])
                # else:
                #     raise ValueError(f"Unable to find an X for boundary vertex {v.index}")
            else:
                all_X.append(self.dict_vert2heedges[(v.index, v.link_edges[0].other_vert(v).index)])
        # all_X = np.array(all_X)
        self.vertex_attributes["X"] = all_X 

    def calculate_angle_sum(self):
        if self.internal_angles is None:
            self._calculate_corner_angles_and_face_areas()
        angle_sum = np.zeros(len(self.verts))
        for i, v in enumerate(self.verts):
            s = 0
            for fc in self.vert2facecorner[v]:
                s += self.internal_angles[fc.index]
            angle_sum[i] = s

        self.vertex_attributes["angle_sum"] = angle_sum

    def calculate_rescaled_factor(self):
        if "angle_sum" not in self.vertex_attributes:
            self.calculate_angle_sum()
        self.vertex_attributes["s"] = 2 * np.pi / self.vertex_attributes["angle_sum"]
        boundary_bool = np.array([v.is_boundary for v in self.verts])
        self.vertex_attributes["s"][boundary_bool] = 1

    def calculate_rescaled_angle_from_X(self):
        for i, v in enumerate(self.verts):
            heX = self.vertex_attributes["X"][i]
            he_current = heX
            scale_factor = self.vertex_attributes["s"][v.index]
            counter = 0
            summed_angle = 0.0
            n_adj_faces = len(v.link_faces)
            # print("//////////////////////")
            # print(v)
            # print(self.vertex_attributes["X"][v.index])
            # while he_current.next.next.twin != heX:
            while he_current != heX or counter == 0:
                if counter == n_adj_faces + 1:
                    raise ValueError("Infinite loop detected")
                counter+=1

                he_current.angle_from_X = summed_angle * scale_factor
                facecorner = self.face_vertex_to_facecorner[he_current.face][v]
                summed_angle += self.internal_angles[facecorner.index]
                # print(he_current)
                # print(he_current.face)
                # print(he_current.angle_from_X)
                # print(summed_angle)
                # print(he_current.next.next.twin)
                # print()
                he_current = he_current.next.next.twin
                if he_current.face is None:
                    he_current.angle_from_X = summed_angle * scale_factor

                    # print(f"breaking at v={v.index}")
                    break
                # if he_current is None:
                #     raise NotImplementedError("Do something")
                
    def calculate_transform_coefficient(self):
        # compute the r_ij coeff
        for k in self.dict_vert2heedges:
            he_ij = self.dict_vert2heedges[k]
            he_ji = he_ij.twin
            theta_i = he_ij.angle_from_X
            # why -pi below : assume that he_ij is the distinguished vector X of vertex vi (he_ij == vi.X) and similary he_ji==vj.X
            # Then the angles theta_i and theta_j are both 0, yet vi.X = -vj.X
            # So subtract pi to calculate the angle theta_j to he_ij from vj's perspective (instead of he_ji from vj's perspective)
            theta_j = he_ji.angle_from_X - np.pi 
            rho_ij = theta_j - theta_i
            he_ij.transport_coeff = rho_ij

    def calculate_holonomy(self):
        holonomy = []
        for f in self.faces:
            vi, vj, vk = f.verts # ccw order
            ii, ij, ik = vi.index, vj.index, vk.index
            # heij, hejk, heki = self.dict_vert2heedges[(ii, ij)], self.dict_vert2heedges[(ij, ik)], self.dict_vert2heedges[(ik, ii)]
            alpha_i = self.internal_angles[self.face_vertex_to_facecorner[f][vi].index]
            alpha_j = self.internal_angles[self.face_vertex_to_facecorner[f][vj].index]
            alpha_k = self.internal_angles[self.face_vertex_to_facecorner[f][vk].index]
            si, sj, sk = self.vertex_attributes["s"][ii], self.vertex_attributes["s"][ij], self.vertex_attributes["s"][ik]
            omega_ijk = alpha_i *(si-1) + alpha_j * (sj-1) + alpha_k * (sk-1) # identical to he_ij.r + he_jk.r + he_ki.r (up to 2pi) with r = transport_coeff
            holonomy.append(omega_ijk)
        self.face_attributes["holonomy"] = np.array(holonomy)


        

       


obj = bpy.data.objects["Suzanne"]
# obj = bpy.data.objects["Torus"]
obj = bpy.data.objects["Torus.001"]

bpy.ops.object.mode_set(mode='OBJECT')
mesh = obj.data
bm = MYMesh()
bm.from_mesh(mesh)
bm.ensure_lookup_tables()
bm.create_halfedge_datastructure()
bm.calculate_angle_sum()
bm.calculate_rescaled_factor()
bm.assign_distinguished_vector_X()
bm.calculate_rescaled_angle_from_X()
bm.calculate_transform_coefficient()
bm.calculate_holonomy()



0.16666666666666666

In [99]:

def calc(self:MYMesh, n, s):

    def f1f2(x):
        x2 = x*x
        x4 = x2*x2
        x5 = x4 * x
        ejx = np.cos(x) + 1j * np.sin(x)
        x4_inv = 1/x4
        jx = 1j * x
        f1 = x4_inv * (3 + jx + x4/24 + 1j*x5/60 + ejx * (-3 + 2 * jx + 0.5*x2))
        f2 = x4_inv * (4 + jx - jx*x2/6 - x4/12 + 1j*x5/30 + ejx * (-4 + 3*jx + x2))
        return f1, f2
    
    def compute_delta_ii(area, nomega, l01, l12, l20, dot0120):
        # lxy = length squared of e_{(i+x)%3, (i+y)%3}
        # exy : vertex[(i+y)%3].co - vertex[(i+x)%3].co
        #dot0120 : e01 dot e20
        return (1/(4*area)) * (l12 + (1/90)*nomega*nomega*(l01 - dot0120 + l20))

    def compute_delta_jk(r12_conj, area, l01, l20, f1, dot0120, f2):
        # lxy = length squared of e_{(i+x)%3, (i+y)%3}
        # exy : vertex[(i+y)%3].co - vertex[(i+x)%3].co
        #dot0120 : e01 dot e20
        return (r12_conj/area) * ((l01 + l20) * f1 - dot0120 * f2)

    secint = SectionIntegrals()

    n_omegas_ijk = n * self.face_attributes["holonomy"]
    ratio_om = compute_mass_ij_direct(n_omegas_ijk)
    f1_nomegas, f2_nomegas = f1f2(n_omegas_ijk)
    area_time_ratio = self.face_areas * ratio_om

    row_m, col_m, val_m = [], [], [] # coo for mass
    row_e, col_e, val_e = [], [], [] # and energy
    for face_index, f in enumerate(self.faces):
        vi, vj, vk = f.verts
        vertex_indices = [vi.index, vj.index, vk.index]
        he = [self.dict_vert2heedges[(vi.index, vj.index)], self.dict_vert2heedges[(vj.index, vk.index)], self.dict_vert2heedges[(vk.index, vi.index)]]

        eij, ejk, eki = vj.co-vi.co, vk.co-vj.co, vi.co-vk.co
        lij2 = eij[0]*eij[0] + eij[1]*eij[1] + eij[2]*eij[2]
        ljk2 = ejk[0]*ejk[0] + ejk[1]*ejk[1] + ejk[2]*ejk[2]
        lki2 = eki[0]*eki[0] + eki[1]*eki[1] + eki[2]*eki[2]
        pn2 = [lij2, ljk2, lki2]
        p = [eij, ejk, eki]
        dot_products = [np.dot(eij, ejk), np.dot(ejk, eki), np.dot(eki, eij)]

        rc = n * np.array([he[0].transport_coeff, he[1].transport_coeff, he[2].transport_coeff]) # MULTIPLIED BY N
        rc = np.conjugate(np.cos(rc) + 1j * np.sin(rc))

        rhs = ratio_om[face_index]
        A = self.face_areas[face_index]
        om = n * self.face_attributes["holonomy"][face_index] # MULTIPLIED BY N

        Mii = A * secint.mass_ii()
        row_m.extend([i, j, k])
        col_m.extend([i, j, k])
        val_m.extend([Mii, Mii, Mii])

        Mij = A * secint.mass_ij(om)
        


        i,j,k = vertex_indices
        he_ij, he_jk, he_ki = self.dict_vert2heedges[(i,j)], self.dict_vert2heedges[(j,k)], self.dict_vert2heedges[(k,i)]
        nrho_ij, nrho_jk, nrho_ki = n * he_ij.transport_coeff, n * he_jk.transport_coeff, n * he_ki.transport_coeff
        r_conj_ij, r_conj_jk, r_conj_ki = np.cos(nrho_ij) - 1j*np.sin(nrho_ij), np.cos(nrho_jk) - 1j*np.sin(nrho_jk), np.cos(nrho_ki) - 1j*np.sin(nrho_ki)

            # MASS 
        # diagonal elements
        a6 = area/6.0
        mii, mjj, mkk = a6, a6, a6
        row_m.extend([i,j,k])
        col_m.extend([i,j,k])
        val_m.extend([mii, mjj, mkk])

        # off diagonal elements
        atr_face = area_time_ratio[face_index] 
        mij, mjk, mki = atr_face*r_conj_ij, atr_face*r_conj_jk, atr_face*r_conj_ki
        
        # mij, mjk, mki = area/12.0,  area/12.0,  area/12.0
        row_m.extend([i,j,k])
        col_m.extend([j,k,i])
        val_m.extend([mij, mjk, mki])

        # DELTA
        a4 = 1/(4*area)
        om = n_omegas_ijk[face_index]
        om2 = om*om
        # careful : minus sign because I'm using eki and not eik (cf article §6.1.1)
        delta_ii = a4 * (ljk2 + om2 * (lij2 - np.dot(eij, eki) + lki2) / 90)
        delta_jj = a4 * (lki2 + om2 * (ljk2 - np.dot(ejk, eij) + lij2) / 90)
        delta_kk = a4 * (lij2 + om2 * (lki2 - np.dot(eki, ejk) + ljk2) / 90)

        f1, f2 = f1_nomegas[face_index], f2_nomegas[face_index]
        # careful : minus sign because I'm using eki and not eik (cf article §6.1.1)
        delta_ij = (r_conj_ij/area) * ((lki2 + ljk2) * f1 - np.dot(eki, ejk) * f2)
        delta_jk = (r_conj_jk/area) * ((lij2 + lki2) * f1 - np.dot(eij, eki) * f2)
        delta_ki = (r_conj_ki/area) * ((ljk2 + lij2) * f1 - np.dot(ejk, eij) * f2)


        # original code from paper : snKii = (s*(om/area))*(Mii) with Mii = area/6
        snKii = s * om / 6.0
        aii, ajj, akk = delta_ii - snKii, delta_jj - snKii, delta_kk - snKii

        omdarea = om/area
        aij = delta_ij - s*(omdarea*mij - 0.5 * 1j * r_conj_ij)
        ajk = delta_jk - s*(omdarea*mjk - 0.5 * 1j * r_conj_jk)
        aki = delta_ki - s*(omdarea*mki - 0.5 * 1j * r_conj_ki)
        row_e.extend([i,j,k,i,j,k])
        col_e.extend([i,j,k,j,k,i])
        val_e.extend([aii, ajj, akk, aij, ajk, aki])


    mass_matrix = scipy.sparse.coo_array((val_m, (row_m, col_m))).tocsr()
    mass_matrix = 0.5 * (mass_matrix.T + mass_matrix)
    energy_matrix = scipy.sparse.coo_array((val_e, (row_e, col_e))).tocsr()

    energy_matrix = energy_matrix + 1e-9 * mass_matrix

    return mass_matrix, energy_matrix


def smallest_eigenvector_square(energy_matrix, mass_matrix, n_iterations):
    """
    Compute the smallest eigenvector of the square using an iterative method.

    Parameters:
    - energy_matrix: Sparse energy matrix (scipy.sparse.csr_matrix)
    - mass_matrix: Sparse mass matrix (scipy.sparse.csr_matrix)
    - n_iterations: Number of iterations for the method.

    Returns:
    - x: Approximation of the smallest eigenvector.
    """
    N = energy_matrix.shape[0]  # Number of rows
    u = np.random.rand(N)       # Random initial vector
    x = u.copy()                # Initialize x

    for _ in range(n_iterations):
        # Solve the linear system: energy_matrix * x = mass_matrix * u
        x = spsolve(energy_matrix, mass_matrix @ u)

        # Re-normalize x with respect to the mass matrix
        x = normalize(x, mass_matrix)
        # x = x/np.absolute(x)

        # Update u for the next iteration
        u = x.copy()

    return x

def normalize(x, mass_matrix):
    """
    Normalize vector x with respect to the mass matrix.
    """
    norm = np.sqrt(x @ (mass_matrix @ x))  # Quadratic form norm
    return x / norm if norm != 0 else x


In [100]:
n = 1
mass_matrix, energy_matrix = calc(bm, n, s=0.0)
# m_inv = scipy.sparse.linalg.inv(mass_matrix)
# matrix_A = m_inv @ energy_matrix

In [101]:
eigvec = smallest_eigenvector_square(energy_matrix, mass_matrix, 10)


In [102]:

all_X = []
for i, v in enumerate(bm.verts):
    he = bm.vertex_attributes["X"][i]
    X = np.array(he.twin.vertex.co - he.vertex.co)
    X_perp = np.dot(X, v.normal)
    X = X - X_perp * v.normal
    all_X.append(X/np.linalg.norm(X))

all_X = np.array(all_X)


u = eigvec
a = (np.angle(u.ravel())/n) / bm.vertex_attributes["s"]
field = np.cos(a) + 1j * np.sin(a)

if 'X' in mesh.attributes:
    mesh.attributes.remove(mesh.attributes["X"])
    mesh.attributes.remove(mesh.attributes["angle"])
    
attr = mesh.attributes.new(name="X", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector', all_X.flatten())

attr = mesh.attributes.new(name="angle", type='FLOAT', domain='POINT')
attr.data.foreach_set('value', a.flatten())
    

In [17]:
np.angle(u)

array([[ 2.25143484],
       [-1.21060419],
       [-1.87666346],
       ...,
       [ 2.22849514],
       [-1.84849651],
       [ 0.21059313]])

In [ ]:
import numpy as np
from scipy.sparse.linalg import spsolve

def smallest_eigenvector_square(energy_matrix, mass_matrix, n_iterations):
    """
    Compute the smallest eigenvector of the square using an iterative method.

    Parameters:
    - energy_matrix: Sparse energy matrix (scipy.sparse.csr_matrix)
    - mass_matrix: Sparse mass matrix (scipy.sparse.csr_matrix)
    - n_iterations: Number of iterations for the method.

    Returns:
    - x: Approximation of the smallest eigenvector.
    """
    N = energy_matrix.shape[0]  # Number of rows
    u = np.random.rand(N)       # Random initial vector
    x = u.copy()                # Initialize x

    for _ in range(n_iterations):
        # Solve the linear system: energy_matrix * x = mass_matrix * u
        x = spsolve(energy_matrix, mass_matrix @ u)

        # Re-normalize x with respect to the mass matrix
        x = normalize(x, mass_matrix)

        # Update u for the next iteration
        u = x.copy()

    return x

def normalize(x, mass_matrix):
    """
    Normalize vector x with respect to the mass matrix.
    """
    norm = np.sqrt(x @ (mass_matrix @ x))  # Quadratic form norm
    return x / norm if norm != 0 else x


In [ ]:
mass_matrix

In [52]:
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt
def mysolve(mass_matrix, energy_matrix):

    N = mass_matrix.shape[0]
    u = np.random.rand(N)-.5 + 1j * (np.random.rand(N)-.5)

    u = u/np.absolute(u)
    norm_log = []
    for i in range(50):
        u = spsolve(energy_matrix, mass_matrix.dot(u))
        # u = u / np.abs(u)
        norm = np.sqrt(u @ (mass_matrix @ u))
        norm_log.append(norm)
        u = u/norm
    print(energy_matrix.dot(u)/mass_matrix.dot(u))
    norm_log = np.array(norm_log)
    return norm_log
norm_log = mysolve(mass_matrix, energy_matrix)

[-4.89515396e+01+1.35603494e+02j -2.07191299e-01+4.67299186e-01j
 -5.11335682e+01+1.34505777e+02j ...  3.24296233e-06+3.71169464e-03j
  6.29154191e-05+3.66674591e-03j  4.57526112e-04+2.74437187e-03j]
